In [ ]:
import matplotlib.pyplot as plt
from ase.optimize.sciopt import *               
from ase.utils.geometry import *
from ase.lattice.spacegroup import crystal
from ase.visualize import *
from ase.lattice.surface import surface
from ase import Atoms
from ase import io 
from ase.io.cif import read_cif
from ase.io.vasp import write_vasp
from abtem.visualize import show_atoms
from ase.visualize.plot import plot_atoms
from ase.build import add_adsorbate
from ase.io.lammpsdata import write_lammps_data
from ase import neighborlist
from ase.build import molecule
from scipy import sparse
from ase.io.proteindatabank import write_proteindatabank
import subprocess
from ase.io import read

In [ ]:
! rm *.vasp data.* *lammpstrj *.lmp log* *.tcl in.FAPI*

# FAPbI3 structures are taken from: http://dx.doi.org/10.1021/ic401215x

### below FA are placed to rectify unit cells

## cubic polymorph

In [ ]:
structure = io.read('cubic_3R.cif')
structure = sort(structure)
tmp_molecule=[]
j = 0
num_atoms = len(structure.get_chemical_symbols())
i = 0
del_index = []
N_index = []
flag = False

while i < num_atoms:
    if(structure.get_chemical_symbols()[i] == 'C'):
        del_index.append(i)
        molecule = io.read('FA.pdb')
        molecule.set_cell(structure.cell)
        xmin = molecule.get_center_of_mass()[0]
        xmax = structure.positions[i, 0]
        ymin = molecule.get_center_of_mass()[1]
        ymax = structure.positions[i, 1]
        zmin = molecule.get_center_of_mass()[2]
        zmax = structure.positions[i, 2]
        molecule.positions += (xmax - xmin, ymax - ymin, zmax - zmin)   
        if j==0:
            tmp_molecule = molecule 
        else:
            tmp_molecule += molecule
        j = j+1
    i = i + 1
    
k = 0
while k < num_atoms:
    if(structure.get_chemical_symbols()[k] == 'N'):
        del_index.append(k)
    k = k + 1

del structure[del_index]  


FA_replaced_structure = structure + tmp_molecule
FA_replaced_structure = sort(FA_replaced_structure)

view (FA_replaced_structure)
FA_replaced_structure

### prepare lammps input file and run

In [ ]:
unit_cell = FA_replaced_structure
rep1 = 3
rep2 = 3
rep3 = 2
i = 0
j = 0
k = 0
x = 0 
y = 0

num = len(unit_cell.get_chemical_symbols())
while k < num:
    cutOff = neighborlist.natural_cutoffs(unit_cell)
    nl = neighborlist.NeighborList(cutOff, self_interaction=False, bothways=True)
    nl.update(unit_cell)
    if(unit_cell.get_chemical_symbols()[k] == 'H'):
        x = nl.get_neighbors(k)
        y = x[0]
        for j in y:
            if(unit_cell.get_chemical_symbols()[j] == 'C'):
                unit_cell.symbols[k] = 'He'          
    k = k + 1
    
supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)
view(supercell)
write_proteindatabank('slab.pdb',supercell, write_arrays=True)

# vmd tcl script for generating lammps data file

In [ ]:
with open("solute.tcl","w") as f:
    print("""
#!/usr/bin/tclsh
if {[catch {package require topotools 1.1} ver]} {
  vmdcon -error "$ver. This script requires at least TopoTools v1.1. Exiting..."
    quit
}
if {[catch {package require pbctools 2.3} ver]} {
  vmdcon -error "$ver. This script requires at least pbctools v2.3. Exiting..."
    quit
}
set fname slab.pdb
# check for presence of coordinate file
if {! [file exists $fname]} {
  vmdcon -error "Required file '$fname' not available. Exiting..."
    quit
}

mol new $fname autobonds no waitfor all

proc distance {new_pos x y z} {
  set x1 [lindex $new_pos 0]
    set bc [pbc get]
    set lx [lindex $bc 0 0]
    set ly [lindex $bc 0 1]
    set lz [lindex $bc 0 2]
    set y1 [lindex $new_pos 1]
    set z1 [lindex $new_pos 2]
    set dx [expr $x - $x1]
    set dy [expr $y - $y1]
    set dz [expr $z - $z1]
    if {$dx >   $lx * 0.5} { set dx [expr $dx - $lx] }
  if {$dx <= -$lx * 0.5} { set dx [expr $dx + $lx] }
  if {$dy >   $ly * 0.5} { set dy [expr $dy - $ly] }
  if {$dy <= -$ly * 0.5} { set dy [expr $dy + $ly] }
  if {$dz >   $lz * 0.5} { set dz [expr $dz - $lz] }
  if {$dz <= -$lz * 0.5} { set dz [expr $dz + $lz] }
  set dr [expr ($dx)**2+($dy)**2+($dz)**2]
    return $dr
}
############### SOLUTE ##################
set selH [atomselect top {name H}]
$selH set type H
$selH set mass 1.00800
$selH set charge 0.4648
set numH [$selH num]
set posH [$selH get {x y z}]
set indexH [$selH get index]

set selHe [atomselect top {name He}]
$selHe set type He
$selHe set mass 1.00800
$selHe set charge 0.22250000 
set numHe [$selHe num]
set posHe [$selHe get {x y z}]
set indexHe [$selHe get index]

set selC [atomselect top {name C}]
$selC set type C
$selC set mass 12.0100
$selC set charge 0.56710000 
set numC [$selC num]
set posC [$selC get {x y z}]
set indexC [$selC get index]

set selN [atomselect top {name N}]
$selN set type N
$selN set mass 14.0100
$selN set charge -0.82450000 
set numN [$selN num]
set posN [$selN get {x y z}]
set indexN [$selN get index]

set selPb [atomselect top {name Pb}]
$selPb set type Pb
$selPb set mass 207.2
$selPb set charge 2.00
set numPb [$selPb num]
set posPb [$selPb get {x y z}]
set indexPb [$selPb get index]

set selI [atomselect top {name I}]
$selI set type I
$selI set mass 126.90
$selI set charge -1.00
set numI [$selI num]
set posI [$selI get {x y z}]
set indexI [$selI get index]
#######################################
set sel [atomselect top all]
set natoms [molinfo top get numatoms]
set bondDistanceHN 1.2
set bondDistanceNC 2.5
set bondDistanceCHa 1.5
set selText all
set selTemp [atomselect top $selText]
set pos [$selTemp get {x y z}]
set index [$selTemp get index]
$selTemp delete
set bondDistance2HN [expr $bondDistanceHN*$bondDistanceHN]
set bondDistance2NC [expr $bondDistanceNC*$bondDistanceNC]
set bondDistance2CHa [expr $bondDistanceCHa*$bondDistanceCHa]
set blist {}

############ SOLUTE ##################
foreach r $posC ind1 $indexC {
# Select neighboring atoms.
  foreach {x y z} $r {break}
  foreach new_pos $posHe ind2 $indexHe {
    set dr [ distance $new_pos $x $y $z ]
      if {($dr < $bondDistance2CHa)} { lappend blist [list $ind1 $ind2]}
  }
}
foreach r $posN ind1 $indexN {
# Select neighboring atoms.
  foreach {x y z} $r {break}
  foreach new_pos $posH ind2 $indexH {
    set dr [ distance $new_pos $x $y $z ]
      if {($dr < $bondDistance2HN)} { lappend blist [list $ind1 $ind2]}
  }
}
foreach r $posN ind1 $indexN {
# Select neighboring atoms.
  foreach {x y z} $r {break}
  foreach new_pos $posC ind2 $indexC {
    set dr [ distance $new_pos $x $y $z ]
      if {($dr < $bondDistance2NC)} { lappend blist [list $ind1 $ind2]}
  }
}

return $blist
set reD {}
foreach b $blist {
  set bPerm [list [lindex $b 1] [lindex $b 0]]
    set match [lsearch $reD $bPerm]
    if {$match == -1} {lappend reD $b}
}
return $reD
topo setbondlist type $reD
topo retypebonds 
vmdcon -info "assigned [topo numbondtypes] bond types to [topo numbonds] bonds:"
vmdcon -info "bondtypes: [topo bondtypenames]"
topo guessangles  
vmdcon -info "assigned [topo numangletypes] angle types to [topo numangles] angles:"
vmdcon -info "angletypes: [topo angletypenames]"
topo guessdihedrals
vmdcon -info "assigned [topo numdihedraltypes] dihedral types to [topo numdihedrals] dihedrals:"
vmdcon -info "dihedraltypes: [topo dihedraltypenames]"
topo guessimpropers
vmdcon -info "assigned [topo numimpropertypes] improper types to [topo numimpropers] impropers:"
vmdcon -info "impropertypes: [topo impropertypenames]"
mol reanalyze top
pbc get
# we use a high-level tool from to multiply the system.
#TopoTools::replicatemol top 4 4 4

# and write out the result as a lammps data file.
topo writelammpsdata data.FAPI full
#topo writegmxtop topol.top

# for easier testing and visualization, we
# also write out copies in .pdb and .psf format.
#animate write pdb solvated.pdb
#animate write psf solvated.psf
# final sanity check the whole system has to be neutral.
$sel delete
set sel [atomselect top all]
set totq [vecsum [join [$sel get charge] { }]]
if {[expr {abs($totq)}] > 0.0005} {
  vmdcon -warning "Total system not neutral: $totq"
}

# done. now exit vmd
quit
""",file=f)

In [ ]:
subprocess.run("vmd -dispdev text -e solute.tcl",shell=True)

# write the force fields file

In [ ]:
with open("in.FAPI","w") as f:
    print("""
bond_style          harmonic
angle_style         harmonic
dihedral_style      opls 
improper_style      cvff
pair_style          hybrid lj/cut/coul/long 10.0 8.0 buck/coul/long 10.0 8.0
kspace_style        pppm 1e-4
special_bonds       amber
variable x          equal 0.966 
variable xA         equal 0.9666
variable APb        equal "70359906.629702/v_xA"
variable rPb        equal "0.131258*v_x"
variable rhoPb      equal "0.0*v_x"
variable xR         equal  0.9666 
variable xAR        equal  0.6
variable APbI       equal "103496.133010/v_xAR"
variable rPbI       equal "0.321737*v_xR"
variable rhoPbI     equal "0.0*v_xR"
variable xIR        equal 0.92 
variable xAIR       equal 1.2
variable AI         equal "22793.338582/v_xAIR"
variable rI         equal "0.482217*v_xIR"
variable rhoI       equal "696.94952*v_xIR"

# Force field
pair_coeff 1 1 lj/cut/coul/long     0.076           3.55                 #C-C
pair_coeff 2 2 lj/cut/coul/long     0.0157          1.06910              #H-H
pair_coeff 3 3 lj/cut/coul/long     0.030           2.50                 #Ha-Ha
pair_coeff 4 4 buck/coul/long       ${AI}           ${rI}      ${rhoI}   #I-I
pair_coeff 5 5 lj/cut/coul/long     0.1700          3.25000              #N-N
pair_coeff 6 6 buck/coul/long       ${APb}          ${rPb}     ${rhoPb}  #Pb-Pb
pair_coeff 2 4 lj/cut/coul/long     0.0574          2.75000              #I-H
pair_coeff 1 4 buck/coul/long       112936.714213   0.342426   0.000000  #I-C
pair_coeff 3 4 lj/cut/coul/long     0.0574          3.10000              #I-Ha
pair_coeff 4 5 buck/coul/long       112936.714213   0.342426   0.000000  #I-N
pair_coeff 4 6 buck/coul/long       ${APbI}         ${rPbI}    ${rhoPbI} #Pb-I
pair_coeff 5 6 buck/coul/long       32690390.937995 0.150947   0.000000  #Pb-N
pair_coeff 2 6 lj/cut/coul/long     0.0140          2.26454              #Pb-H
pair_coeff 1 6 buck/coul/long       32690390.937995 0.150947   0.000000  #Pb-C
pair_coeff 3 6 lj/cut/coul/long     0.0140          2.70999              #Pb-Ha

#Bond Coeffs
bond_coeff    1   340.4000   1.0800     #C-Ha
bond_coeff    2   448.0000   1.3650     #C-N
bond_coeff    3   434.0000   1.0100     #H-N

#Angle Coeffs
angle_coeff    1   35.000    120.000    #C-N-H    
angle_coeff    2   35.000    113.000    #H-N-H    
angle_coeff    3   35.000    119.100    #Ha-C-N
angle_coeff    4   70.000    120.000    #N-C-N
 
# Dihedral coefficients
dihedral_coeff   1   0.000      3.900      0.000      0.000    #Ha-C-N-H 
dihedral_coeff   2   0.000      3.900      0.000      0.000    #H-N-C-N

#impropers
improper_coeff   1   2.500      -1       2      #C-N-N-Ha 
improper_coeff   2   2.500      -1       2      #N-C-H-H
""",file=f)

# lammps start file with parameters

In [ ]:
with open("start.lmp","w") as f:
    print("""
dimension       3
boundary        p p p
units           real
atom_style      full
variable        seed world 1428
variable        temperature equal 300.0
variable        temperature2 equal 300.0
variable        tempDamp equal 100.0 # approx 0.1 ps
variable        pressure equal 1.00
variable        pressureDamp equal 500.0
variable        freq equal 500
read_data       data.FAPI
include         in.FAPI
thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz

# Minimization
min_style cg
#fix 1 all box/relax aniso 0.0 vmax 0.105
minimize        1.0e-4 1.0e-6 1000 10000
#unfix 1
write_data      data.min

# NVT
dump            myDump1 all atom 500 out.0.lammpstrj 
fix             1 all temp/csvr ${temperature} ${temperature} ${tempDamp} ${seed}
fix             2 all shake 0.0001 20 100000 t 2 3
fix             3 all nve
timestep        2.0
velocity        all create ${temperature} ${seed} dist gaussian
run             10000
unfix           1
unfix           2
unfix           3
write_data      data.NVT
undump          myDump1
reset_timestep  0
# NPT
dump            myDump2 all atom 500 out.1.lammpstrj 
fix             1 all temp/csvr ${temperature} ${temperature} ${tempDamp} ${seed}
fix             2 all shake 0.0001 20 100000 t 2 3
fix             3 all nph tri ${pressure} ${pressure} ${pressureDamp} 
fix             4 all momentum 10000 linear 1 1 1
run             50000
unfix           1
unfix           2
unfix           3
unfix           4
undump          myDump2
reset_timestep  0
write_restart   restart.file
write_data      data.eq
""",file=f)

# run lammps

In [ ]:
subprocess.run("nohup mpirun -np 20 lmp_intel_cpu_intelmpi < start.lmp ",shell=True)

# view trajectories

In [ ]:
traj = read('out.1.lammpstrj', index=":", parallel=True)
view(traj)